In [0]:
!pip install openai langchain databricks_langchain

  Obtaining dependency information for databricks_langchain from https://files.pythonhosted.org/packages/cb/d4/cd43f51227f9cb892e242b81d1bed18daf1cd0ce6be66d9ed0a1456460ba/databricks_langchain-0.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for databricks-ai-bridge>=0.3.0 from https://files.pythonhosted.org/packages/94/99/54683ba8f2b66bc8f80d37af87702d3f6f5840fe1d995882e66f5025d74d/databricks_ai_bridge-0.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for databricks-vectorsearch>=0.40 from https://files.pythonhosted.org/packages/cb/07/bb8ef2f2abd4f5081273c879cb884f63f57e61c44c761a17da418e8be2b9/databricks_vectorsearch-0.50-py3-none-any.whl.metadata
  Obtaining dependency information for mlflow>=2.20.1 from https://files.pythonhosted.org/packages/b8/f3/89dc9daf896ec5caea037dfc8cd24bcef69995aed2d30164afd69af5ec8d/mlflow-2.20.3-py3-none-any.whl.metadata
  Obtaining dependency information for unitycatalog-langchain from https://files.pythonhosted.org/pac

## **LLMChain with our zero-shot prompt input**

In [0]:
# Step:01 Make Prompt Template
from langchain import PromptTemplate
demo_template='''I want you to act as a Generative AI expert for people.
In an easy way, explain the basics of {topic_name}.'''
prompt=PromptTemplate(
    input_variables=['topic_name'],
    template=demo_template
    )
prompt.format(topic_name='Indexing')


# Step:02 Call LLM model and store in One object

from databricks_langchain import ChatDatabricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-1-70b-instruct", 
    temperature=0.1,
    max_tokens=250)

# Step:03 Cretae Chain
from langchain.chains import LLMChain
chain1 = LLMChain(llm=llm,prompt=prompt)

# Step:04 call LLM for specific topic
response=chain1.invoke('RAG')
print(response['text'])

I'd be happy to explain the basics of RAG in simple terms.

RAG stands for "Retrieval-Augmented Generation". It's a type of artificial intelligence (AI) technology that helps generate human-like text or responses.

**What does RAG do?**

Imagine you're having a conversation with a chatbot or a virtual assistant. You ask it a question, and it responds with a relevant answer. But, have you ever wondered how it comes up with that answer?

RAG is a technique that helps the AI system generate more accurate and informative responses by:

1. **Retrieving** relevant information from a large database or knowledge base.
2. **Augmenting** that information with its own understanding and generation capabilities.

**How does RAG work?**

Here's a simplified overview:

1. **User Input**: You ask a question or provide a prompt to the AI system.
2. **Retrieval**: The AI system searches its database or knowledge base to find relevant information related to your input.
3. **Augmentation**: The retrieved 

## **Simple language translation Template**

In [0]:

%python
# Step:01 Make Prompt Template

from langchain import PromptTemplate
template = ''' In an easy way translate the following sentence '{sentence}' into {target_language}'''
prompt = PromptTemplate(
    input_variables=['sentence', 'target_language'],
    template=template,
)
prompt.format(sentence="What is your name?", target_language='Marathi')

# Step:02 Call LLM model and store in One object

from databricks_langchain import ChatDatabricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-1-70b-instruct", 
    temperature=0.6,
    max_tokens=250
)

# Step:03 Create Chain
from langchain.chains import LLMChain
chain2 = LLMChain(llm=llm, prompt=prompt)


# Step:04 call LLM for specific topic
response = chain2({'sentence': "Hello, what is your name", 'target_language': 'Marathi'})
response['text']

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4406769114426620>, line 3
      1 # Step:01 Make Prompt Template
----> 3 from langchain import PromptTemplate
      4 template = ''' In an easy way translate the following sentence '{sentence}' into {target_language}'''
      5 prompt = PromptTemplate(
      6     input_variables=['sentence', 'target_language'],
      7     template=template,
      8 )

ModuleNotFoundError: No module named 'langchain'

## **Few-Shot Prompt Template**

In [0]:
from langchain import PromptTemplate,FewShotPromptTemplate
examples = [{"word": "fast", "synonym": "quick"},
                     {"word": "hard", "synonym": "difficult"}]

example_formatter_template = """Word: {word}
Synonym: {synonym}
"""
example_prompt = PromptTemplate(
    input_variables=["word", "synonym"],
    template=example_formatter_template,
)

In [0]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the synonym of every input\n",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nSynonym: ",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [0]:
print(few_shot_prompt.format(input='Elegant'))

Give the synonym of every input

Word: fast
Synonym: quick

Word: hard
Synonym: difficult

Word: Elegant
Synonym: 


In [0]:
chain=LLMChain(llm=llm,prompt=few_shot_prompt)
chain({'input':"high"})

{'input': 'high',
 'text': 'The pattern continues...\n\nWord: high\nSynonym: tall'}